# Defra Earth Observation Data Service API

# Introduction 
This notebook has been created to introduce and test the Defra Earth Observation Data Service API.  Follow the three steps below to import libraries and configuration file, apply query parameters to filter available data, and create a parameterised URL using the api/layers endpoint with query parameters for date, geometry and satellite.  

## 1) Import libraries and configuration file
Here we *import* the libraries required to run this notebook.
If there are any errors when running this cell (all cells can be run using *'Shift-Return'*) then check that the libraries were correctly installed through **Conda** or the package manager that you use.

Before continuing with this notebook, ensure that the **USERNAME** and **ACCESS_TOKEN** parameters in the _config.py_ file relate to the username to be used for the following commands.

Check all other _config.py_ parameters are as you would expect them, especially the URL and GEOM parameters.

In [11]:
# Import general and geospatial data handling libraries
import urllib3, requests, os, importlib, json
import logging
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import Image
from datetime import datetime
import time
import os
import pathlib
from pathlib import Path
# Import your configuration file
import config
# configure specific library calls
importlib.reload(config)

<module 'config' from 'C:\\Users\\Kostas.Sideris\\config.py'>

In [12]:
# initialise notebook
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.options.display.max_colwidth = 100

output_fmt='%Y%m%dT%H%M%S'
pretty_fmt='%Y-%m-%d %H:%M:%S'

# create output directory if it does not exist
output_dir = Path(Path.cwd() / 'data')
if not output_dir.is_dir():
    output_dir.mkdir()

## 2) Configure query parameters
The next cell sets a series of query parameters for searching through the data on the system. You can change these and re-run the cell to investigate different scenarios. 

In [19]:
# configure variables
DATE_RANGE_START = '2019-01-01'
DATE_RANGE_END = '2019-06-01'
SATELLITE_ID = 2
TITLE = 'S2A'
TYPE = 'raster'
GEOM = "POLYGON((-0.09799 51.49697,-0.08799 51.49697,-0.08799 51.48697,-0.09799 51.48697,-0.09799 51.49697))"
# User should select APPLY_MIN_CLOUD_FUNCTION == True with APPLY_FILTERS == True
APPLY_FILTERS = True
APPLY_MIN_CLOUD_FUNCTION = True

# search locations
# England-wide footprint
# GEOM = "POLYGON((-3.70583842728179391 55.44991031769518486, -2.13623226289823265 55.44991031769518486, -0.53426308481603968 55.43372881084587078, -0.58280760536398546 54.54374593413353978, -0.63135212591193124 53.65376305742121588, 0.84116499737574024 53.58903703002395247, 2.31368212066341172 53.65376305742121588, 2.3622266412113575 52.76378018070888487, 2.39458965490998743 51.84143429029792571, 2.39458965490998743 50.90290689303765248, 2.3622266412113575 50.07765004372258488, 0.9867985590195758 50.04528703002395673, 0.53371636723875326 49.9967425094760074, -0.82553020810371081 50.04528703002395673, -2.23332130399412243 50.06146853687327081, -3.60874938618590413 50.04528703002395673, -5.42107815330919074 49.9967425094760074, -6.82886924919960148 50.04528703002395673, -5.50198568755576645 50.17473908481847644, -5.40489664645987578 50.87054387933902433, -3.6249308930352182 50.91908839988696656, -3.70583842728179391 55.44991031769518486))"
# small London footprint
# GEOM = "POLYGON((-0.09799 51.49697,-0.08799 51.49697,-0.08799 51.48697,-0.09799 51.48697,-0.09799 51.49697))"

try:
    assert not (APPLY_FILTERS == False and APPLY_MIN_CLOUD_FUNCTION == True)
except AssertionError as err:
    logging.error("\n\n # CONFIG ERROR! \n If you want to use APPLY_MIN_CLOUD_FUNCTION, then set APPLY_FILTERS to 'True'")
    raise err

# do config parameter validation. ## User needs to select APPLY_MIN_CLOUD_FUNCTION == True and SATELLTE_ID == 2
try:
    assert not (APPLY_MIN_CLOUD_FUNCTION == True and SATELLITE_ID == 1)
except AssertionError as err:
    logging.error('\n # CONFIG ERROR \n You are trying to apply the "APPLY_MIN_CLOUD_FUNCTION" but have not set "SATELLTE_ID" to "2"')
    raise err

    

#****NOTES****
#
#APPLY_FILTERS = False >> You download all layers from the EO service
#APPLY_FILTERS = True >> The above filter parameters will be applied to the api query#
#
#APPLY_MIN_CLOUD_FUNCTION = False >> Applies to S2 data only, will download all S2 data as per filter parameter spec
#APPLY_MIN_CLOUD_FUNCTION = True >> Applies to S2 data only, will only download S2 granules that meet 
#    the lowest cloud cover criteria for each unique granule reference per orbit


##  3) Create parameterised URL
The following cell creates a parameterised URL using the API 'layers' endpoint and a set of query parameters. Copy the output and paste it into a browser address bar. The resultant page provides an indication of the information returned to this notebook (based on the input parameters) from the server. 

In [20]:
# Initialise the required variables using the details in your config file (config.py)

# Create URLs for the Geoserver endpoint (gs_end_point), OWS endpoint (serverURL)
gsdownload_template_payload = '<?xml version="1.0" encoding="UTF-8"?><wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd"><ows:Identifier>gs:Download</ows:Identifier><wps:DataInputs><wps:Input><ows:Identifier>layerName</ows:Identifier><wps:Data><wps:LiteralData>@@@</wps:LiteralData></wps:Data></wps:Input><wps:Input><ows:Identifier>outputFormat</ows:Identifier><wps:Data><wps:LiteralData>image/tiff</wps:LiteralData></wps:Data></wps:Input></wps:DataInputs><wps:ResponseForm><wps:ResponseDocument storeExecuteResponse="true" status="true"><wps:Output asReference="true" mimeType="application/zip"><ows:Identifier>result</ows:Identifier></wps:Output></wps:ResponseDocument></wps:ResponseForm></wps:Execute>'
headers = {'Content-type': 'application/xml','User-Agent': 'curl/7.65.1'}
wps_server = config.URL + config.WPS_SUF + '?access_token=' + config.ACCESS_TOKEN

    
# If the APPLY_FILTERS config option is set to True then construct a URL
if APPLY_FILTERS:
    the_url = config.URL + ('/api/layers/?username=' + config.USERNAME 
                            + '&api_key=' + config.ACCESS_TOKEN
                            + '&limit=20000&offset=0'
                            + '&keywords__slug__in=sentinel-' + str(SATELLITE_ID)
                            + '&geometry=' + GEOM
                            + '&date__range=' + DATE_RANGE_START + '%2000:00,' + DATE_RANGE_END + '%2023:59'
                            + '&title__icontains=' + TITLE
                            + '&type__in=' + TYPE
                           )
    
# or if the option is not set to True 
else:
    the_url = config.URL + ('/api/layers/?username=' 
                            + config.USERNAME + '&api_key=' 
                            + config.ACCESS_TOKEN + '&limit=20000&offset=0')


print(the_url)

https://earthobs.defra.gov.uk/api/layers/?username=kostas.sideris&api_key=3FKZcj7oGza5DiZ7KBUCVbhMMYLDTl&limit=20000&offset=0&keywords__slug__in=sentinel-2&geometry=POLYGON((-0.09799 51.49697,-0.08799 51.49697,-0.08799 51.48697,-0.09799 51.48697,-0.09799 51.49697))&date__range=2019-01-01%2000:00,2019-06-01%2023:59&title__icontains=S2A&type__in=raster


##  4) Make a GET request using the URL above and parse the output to a dataframe

In [21]:
# make the request to the API and parse the output to a pandas dataframe
try:
    # parse json response from the geonode api to a pandas dataframe object
    payload = requests.get(url=the_url,headers=headers)

    # create a json object of the api payload content
    the_json = json.loads(payload.content)

    # load json to pandas dataframe
    df_full_layer_list = json_normalize(the_json, 'objects')
except:
    raise Exception('ERROR. The the server response is not correctly formed. Please check you authentication token and internet connection')

if len(df_full_layer_list) == 0:
    raise Exception('ERROR. The API response was empty, no data to process or download, check the filters match data in the EO Portal')

# add granule reference and extract ARCSI_CLOUD_COVER from supplement information value
if SATELLITE_ID == 2:
    df_full_layer_list['granule-ref'] = df_full_layer_list['title'].str.split('_',n=4).str[3]
    df_full_layer_list['orbit-ref'] = df_full_layer_list['title'].str.split('_',n=5).str[-2]
    df_full_layer_list['ARCSI_CLOUD_COVER'] = df_full_layer_list['supplemental_information'].str.split(n=6).str[5]    
    
# show first five rows of the dataframe
df_full_layer_list.head(5)

,abstract,alternate,bbox_x0,bbox_x1,bbox_y0,bbox_y1,category__gn_description,constraints_other,csw_type,csw_wkt_geometry,...,supplemental_information,temporal_extent_end,temporal_extent_start,thumbnail_url,title,typename,uuid,granule-ref,orbit-ref,ARCSI_CLOUD_COVER
0,"These data have been created by the Department for Environment, Food and Rural Affairs (Defra) i...",geonode:S2A_20190601_lat52lon07_T30UYC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,531900.000000000000000,640160.000000000000000,158930.000000000000000,270200.000000000000000,imagerybasemapsearthcover,Other Restrictions,dataset,"POLYGON((-0.108823824666 51.2788586054,-0.108823824666 52.3140704528,1.51913063352 52.3140704528...",...,Data Collection Time: 11:06:21\nARCSI_CLOUD_COVER: 0.3322347402572632\nARCSI_AOT_RANGE_MAX: 0.5\...,2019-06-01T11:06:21,2019-06-01T11:06:21,https://earthobs.defra.gov.uk/uploaded/thumbs/layer-8f393973-dadf-4682-8fc0-6e7e3eeb01b0-thumb.png,S2A_20190601_lat52lon07_T30UYC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,geonode:S2A_20190601_lat52lon07_T30UYC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,8f393973-dadf-4682-8fc0-6e7e3eeb01b0,T30UYC,ORB137,0.3322347402572632
1,"These data have been created by the Department for Environment, Food and Rural Affairs (Defra) i...",geonode:S2A_20190529_lat52lon07_T30UYC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,531900.000000000000000,640160.000000000000000,158930.000000000000000,270200.000000000000000,imagerybasemapsearthcover,Other Restrictions,dataset,"POLYGON((-0.108823824666 51.2788586054,-0.108823824666 52.3140704528,1.51913063352 52.3140704528...",...,Data Collection Time: 10:56:21\nARCSI_CLOUD_COVER: 0.9111020565032959\nARCSI_AOT_RANGE_MAX: 0.5\...,2019-05-29T10:56:21,2019-05-29T10:56:21,https://earthobs.defra.gov.uk/uploaded/thumbs/layer-9354f283-eaf9-4fa7-9a12-9336905357f8-thumb.png,S2A_20190529_lat52lon07_T30UYC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,geonode:S2A_20190529_lat52lon07_T30UYC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,9354f283-eaf9-4fa7-9a12-9336905357f8,T30UYC,ORB094,0.9111020565032959
2,"These data have been created by the Department for Environment, Food and Rural Affairs (Defra) i...",geonode:S2A_20190525_lat52lon075_T30UXC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,431970.000000000000000,540240.000000000000000,160300.000000000000000,271580.000000000000000,imagerybasemapsearthcover,Other Restrictions,dataset,"POLYGON((-1.54241678439 51.3245201724,-1.54241678439 52.3413723269,0.0563450138848 52.3413723269...",...,Data Collection Time: 11:21:21\nARCSI_CLOUD_COVER: 0.5613343119621277\nARCSI_AOT_RANGE_MAX: 0.5\...,2019-05-25T11:21:21,2019-05-25T11:21:21,https://earthobs.defra.gov.uk/uploaded/thumbs/layer-6a13e6e5-3573-46b8-aee5-d1a2eff0a599-thumb.png,S2A_20190525_lat52lon075_T30UXC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,geonode:S2A_20190525_lat52lon075_T30UXC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,6a13e6e5-3573-46b8-aee5-d1a2eff0a599,T30UXC,ORB037,0.5613343119621277
3,"These data have been created by the Department for Environment, Food and Rural Affairs (Defra) i...",geonode:S2A_20190522_lat52lon075_T30UXC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,431970.000000000000000,540240.000000000000000,160300.000000000000000,271580.000000000000000,imagerybasemapsearthcover,Other Restrictions,dataset,"POLYGON((-1.54241678439 51.3245201724,-1.54241678439 52.3413723269,0.0563450138848 52.3413723269...",...,Data Collection Time: 11:06:21\nARCSI_CLOUD_COVER: 0.9316290616989136\nARCSI_AOT_RANGE_MAX: 0.5\...,2019-05-22T11:06:21,2019-05-22T11:06:21,https://earthobs.defra.gov.uk/uploaded/thumbs/layer-3389531c-b884-4f3b-8208-57e07b7b53c8-thumb.png,S2A_20190522_lat52lon075_T30UXC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,geonode:S2A_20190522_lat52lon075_T30UXC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref,3389531c-b884-4f3b-8208-57e07b7b53c8,T30UXC,ORB137,0.9316290616989136
4,"These data have been created by t

##  5) Sort the dataframe by % cloud cover and group by granule name and orbit 
The following cell sorts the dataframe by % cloud cover ascending and groups the results to show the lowest cloud cover per granule, and per granule per relative orbit.  This is important because the imagery with the lowest cloud may be from an orbit where the granule lies on the edge of the swath and therefore does not provide complete coverage of the granule footprint.

In [22]:
if APPLY_MIN_CLOUD_FUNCTION and SATELLITE_ID == 2:
    df_min_cloud_per_granule = df_full_layer_list.sort_values("ARCSI_CLOUD_COVER").groupby(["granule-ref"], as_index=False).first()  
    df_min_cloud_per_granule_per_orbit = df_full_layer_list.sort_values("ARCSI_CLOUD_COVER").groupby(["granule-ref",'orbit-ref'], as_index=False).first()

##  6) View and download the results 

In [23]:
# list the column headers of the 'min cloud per granule' data frame
df_min_cloud_per_granule.columns

Index(['granule-ref', 'abstract', 'alternate', 'bbox_x0', 'bbox_x1', 'bbox_y0',
       'bbox_y1', 'category__gn_description', 'constraints_other', 'csw_type',
       'csw_wkt_geometry', 'data_quality_statement', 'date', 'date_type',
       'default_style', 'detail_url', 'edition', 'geogig_link', 'gtype',
       'has_time', 'id', 'is_approved', 'is_published', 'keywords', 'language',
       'license', 'maintenance_frequency', 'name', 'online', 'owner__username',
       'owner_name', 'popular_count', 'purpose', 'rating', 'regions',
       'resource_uri', 'restriction_code_type', 'share_count',
       'spatial_representation_type', 'srid', 'store_type',
       'supplemental_information', 'temporal_extent_end',
       'temporal_extent_start', 'thumbnail_url', 'title', 'typename', 'uuid',
       'orbit-ref', 'ARCSI_CLOUD_COVER'],
      dtype='object')

In [24]:
# show the 'min cloud per granule' dataframe with just the most relevant columns
df_min_cloud_per_granule[["granule-ref","orbit-ref","date","ARCSI_CLOUD_COVER"]]

,granule-ref,orbit-ref,date,ARCSI_CLOUD_COVER
0,T30UXC,ORB037,2019-02-24T11:21:11,0.029151706025004387
1,T30UYC,ORB094,2019-03-30T10:56:31,0.08980198204517365


In [25]:
# output the minimum cloud lists as a CSV to a the 'data' directory
df_min_cloud_per_granule.to_csv(Path(output_dir / 'min_cloud_per_granule.csv'))
df_min_cloud_per_granule_per_orbit.to_csv(Path(output_dir / 'min-cloud-per-granule-per-orbit.csv'))

# Check the 'data' directory in your Scripts folder, open the two CSV files

In [26]:
# show the list of filenames in the 'minimum cloud per granule' list
list_to_download = df_min_cloud_per_granule.title.tolist()
list_to_download

['S2A_20190224_lat52lon075_T30UXC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
 'S2A_20190330_lat52lon07_T30UYC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref']

In [27]:
# show the list of filenames in the 'minimum cloud per granule per orbit' list
list_to_download_orbit = df_min_cloud_per_granule_per_orbit.title.tolist()
list_to_download_orbit

['S2A_20190224_lat52lon075_T30UXC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
 'S2A_20190330_lat52lon075_T30UXC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
 'S2A_20190221_lat52lon075_T30UXC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
 'S2A_20190330_lat52lon07_T30UYC_ORB094_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
 'S2A_20190422_lat52lon07_T30UYC_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref']